In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft

In [ ]:
import numpy as np

In [ ]:
nu_po_di_ = [
    [[-1], [0], [2]],
    [[-1, -1], [0, 0], [2, 2]],
    [[9, 0], [5, 0], [0, 9], [3, 2], [9, 9]],
    [[0, -1, 8], [1, 0, 16], [2, 4, 24]],
]

nu_po_di_ = [np.array(nu_po_di) for nu_po_di in nu_po_di_]

## get_probability

In [ ]:
for nu_po_di in nu_po_di_:

    kraft.probability.get_probability(nu_po_di)

## get_posterior_probability

In [ ]:
for nu_po_di in nu_po_di_:

    kraft.probability.get_posterior_probability(nu_po_di)

## plot_nomogram